## Traxsource Scraping
1. Create the credential for youtube v3 in your project
2. download the credentials 
3. Add test user 
4. Run code snippets and authenticate in browser

In [2]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.traxsource.com/genre/4/house"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
top10 = soup.find_all("div", {"class": "top-item"})

for track in top10:

    track_node = track

    # Get position of the track
    position = track_node.findAll("div", {"class": "ttib position"})[0].getText()
    position = position.replace('\n', '').strip()

    # Get track infos
    info = track_node.findAll("div", {"class": "ttib info"})[0]
    artist = info.find("a", class_='com-artists').text
    title = info.find("a", class_='com-title').text
    label = info.find("a", class_='com-label').text

    # Print infos
    print('Position: ', position)
    print('Name:     ', title)
    print('Artist:   ', artist)
    print('Label:    ', label)
    print('')

Position:  1
Name:      Do What You Wanna Do (Extended Mix)
Artist:    The Shapeshifters
Label:     Pleased As Punch

Position:  2
Name:      I Thought It Was You (Extended Mix)
Artist:    David Penn
Label:     Central Station Records

Position:  3
Name:      Can U Feel It (2023 Remaster)
Artist:    Jay Vegas
Label:     Hot Stuff

Position:  4
Name:      Don't Pay Me No Mind
Artist:    Mike Dunn
Label:     Nu Groove Records

Position:  5
Name:      Runaway Love (Michael Gray Dub)
Artist:    Bobby D'Ambrosio
Label:     Soulfuric

Position:  6
Name:      Frequency
Artist:    Grant Nelson
Label:     Swing City

Position:  7
Name:      Serious (Wayne Soul Avengerz & Odyssey Inc. Extended Remix)
Artist:    Donna Allen
Label:     High Fashion Music

Position:  8
Name:      Living My Life (Golden) (Extended Mix)
Artist:    Higgo
Label:     Helix Records

Position:  9
Name:      Do You Feel The Same? (Luke Solomon Tribute Remix)
Artist:    Hercules & Love Affair
Label:     Defected

Position: 